In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path as P
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
plt.rcParams['font.size'] = 12  # 字体大小
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
from collections import defaultdict
import plotly.express as px
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow.keras as keras

In [104]:
scaler=pickle.load(P('问题2-MinMaxScaler-训练数据.csv').open('rb'))
# df=pd.read_csv('问题2-训练数据-归一化.csv',index_col='time', parse_dates=True)
df=pd.read_csv('问题2-训练数据-归一化.csv').drop(columns='time')
model=keras.models.load_model('./problem2_checkpoint.h5')

In [105]:
scaler.scale_, scaler.min_

(array([0.08025682, 0.03125977, 0.0195733 , 0.0256476 , 0.00112134,
        0.02069108]),
 array([-0.77367576, -0.92872773, -0.82912507, -0.00692485,  0.        ,
         0.48272295]))

In [99]:
df_raw=pd.read_csv('问题2-原始训练数据.csv')
df_raw

,time,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
0,2012-01-01,13.73,30.87,42.79,0.61,2.29,-22.94
1,2012-02-01,13.00,30.87,42.79,0.86,2.29,-17.86
2,2012-03-01,12.60,30.87,42.79,1.91,21.84,-7.11
3,2012-04-01,11.97,30.73,42.76,7.20,6.10,6.94
4,2012-05-01,14.18,29.99,42.58,18.10,50.80,14.61
...,...,...,...,...,...,...,...
118,2021-11-01,13.91,52.17,93.43,1.44,530.86,-5.89
119,2021-12-01,13.14,52.14,93.45,0.93,193.80,-12.41
120,2022-01-01,12.45,52.14,93.45,0.44,607.31,-17.09
121,2022-02-01,12.10,52.14,93.45,0.27,796.04,-17.93


In [84]:
pd.date_range?

In [70]:
df_test = df.iloc[-3:, :]
test=df_test.values.reshape([1,3,-1])
df_test,test

(     10cm湿度(kg/m2)  40cm湿度(kg/m2)  100cm湿度(kg/m2)  土壤蒸发量(mm)   降水量(mm)  \
 120       0.225522       0.701157        1.000000   0.004360  0.681001   
 121       0.197432       0.701157        1.000000   0.000000  0.892632   
 122       0.426966       0.700844        0.999804   0.146448  0.129593   
 
       平均气温(℃)  
 120  0.129112  
 121  0.111732  
 122  0.433892  ,
 array([[[0.22552167, 0.70115661, 1.        , 0.00436009, 0.68100113,
          0.12911235],
         [0.19743178, 0.70115661, 1.        , 0.        , 0.89263167,
          0.11173184],
         [0.42696629, 0.70084401, 0.99980427, 0.14644781, 0.12959329,
          0.43389199]]]))

In [79]:
def predict(model, df_test: pd.DataFrame, n: int, seqlen: int):
    for i in range(n):
        test=df_test.iloc[-seqlen:, :].values.reshape([1,seqlen,-1])
        pred=model.predict(test)
        df_pred=pd.DataFrame(pred, columns=df.columns)
        df_test = pd.concat([df_test, df_pred])
        
    df_test.index=pd.DatetimeIndex()
    
    return df_test

In [108]:
df_pred=predict(model, df_test, 21, seqlen=3)

In [109]:
pred=scaler.inverse_transform(df_pred)
df_pred=pd.DataFrame(data=pred, columns=df_pred.columns)
df_pred

,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
0,12.450000,52.140000,93.450000,0.440000,607.310000,-17.090000
1,12.100000,52.140000,93.450000,0.270000,796.040000,-17.930000
2,14.960000,52.130000,93.440000,5.980000,115.570000,-2.360000
3,12.576052,47.738105,84.404251,2.069132,458.884764,-13.894871
4,12.110697,48.499003,86.540301,-1.544589,678.515615,-19.987352
5,14.975147,49.804043,88.745572,4.498087,97.953953,-3.475068
6,12.626191,46.577023,79.407803,3.102306,400.890058,-13.221172
7,12.109597,46.725401,82.691493,-2.036123,634.178377,-21.754162
8,14.779870,48.576840,85.722500,4.097910,87.226419,-4.687309
9,12.693471,45.893927,75.723302,3.698036,370.355693,-13.080548


In [91]:
pd.Period(year=2022,month=1,freq='1M')+1

Period('2022-02', 'M')

In [110]:
px.line(df_pred)